In [1]:
!python -m pip install keras-tuner

     |████████████████████████████████| 54 kB 1.3 MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-py3-none-any.whl size=73198 sha256=69b0b949ffac72a9e82a04f1d4f7cc1a77d077501115d3830f62ff70f70f950f
  Stored in directory: /root/.cache/pip/wheels/0b/cf/2f/1a1749d3a3650fac3305a8d7f9237b6de7c41068e2f8520ca2
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=728dbea61f07bcde7d6697c4ed3863f291c020f3a32527226d7c7df1a23e3f5c
  Stored in directory: /root/.cache/pip/wheels/ba/ad/c8/2d98360791161cd3db6daf6b5e730f34021fc9367d5879f497
Successfully built keras-tuner terminaltables
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [3]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Data observations
*Have NaNs:* Age, Fare (some zeros, nans too), Cabin, Embarked
*NOTE:* maybe need to approximate missing values using some other technique, like an additional model?

* (+) Pclass:
  * 1 - 3 number, 1 being the highest
  * Range: 1-3
  * Current approach: normalize by 3
* (+) Name:
  * has person's title, which could be used (Mr, Ms, Mrs, etc.)
  * From title, can infer marital status?
  * Current approach: extract titles, convert them to one-hot, and calculate 'Married' based on title (1 - married (Mr, Mrs), -1 - unmarried (Miss, Master), 0 - unknown (other titles))
  * Potential improvements: use more titles for getting 'married'; use 'maiden name' in calculation of 'married'; use 'nickname' somehow?
* (+) Sex:
  * Either male or female
  * male: 65%, female: 35%
  * Current approach: convert to one-hot.
  * Potential improvements: use 1 and -1 for sexes?
* (+) Age:
  * has fractions if approximated. Has missing values.
  * Range: 0.42-80
  * Current approach: fill NaN with average in group-by Pclass-Sex, but create a column that identifies missing values. Also, normalize by 80.
  * Potential improvements: have a better approximation of age.
* (+) SibSp:
  * how many siblings or spouses on board.
  * Range: 0-8
  * Current approach: normalize by 8.
* (+) Parch:
  * How many parents/children. (can be 0 for babies, if with nannies)
  * Range: 0-6
  * Current approach: normalize by 6.
* Ticket:
  * A number with some optional letters (which can have some meaning?).
  * Has repetitions (maybe for people travelling together).
* (+) Fare:
  * can have zeros (what do they mean?). Can have omitted (just one in test).
  * Range: 0-512.3292
  * Current approach: fill nan with mean, normalize by 512. 
* (+) Cabin:
  * has a lot of omitted values (78%). Can have multiple values (probably for families?).
  * One value is a letter with a number. (both probably have meaning and impact?)
  * Current approach: convert to one-hot (based on letter), include a 'nan' column for those that are missing values. Create a column for cabin number, and a column to identify missing numbers.
  * Potential improvements: maybe cabin number itself doesn't mean much? Also, maybe need to deal with missing values in a different way? Also, maybe deal with multiple values better?
* (+) Embarked:
  * Either of 3 letters (with different frequency). Has just a few omitted.
  * S - 72/65%, C - 19/24%, Q - 9/11%
  * Current approach: convert to one-hot matrix.
  * Potential improvements: somehow take into the account different distribution of embarkation city?

In [5]:
def prepare_data(csv_data):
    # To display all the columns from left to right without breaking into next line.
    pd.set_option('display.width', 1500)

    import re
    
    features = "Pclass Sex SibSp Parch Fare Embarked Name Cabin Age".split()
    
    titles = ['Mr', 'Mrs', 'Miss', 'Master', 'Ms', 'Don', 'Rev', 'Dr', 'Mme', 'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess', 'Jonkheer', 'Dona']
    
    # === Get X - the features. ===

    X = csv_data[features].copy()

    # == Post-process data ==

    if "Pclass" in X:
        X.Pclass = X.Pclass.divide(3)

    if "SibSp" in X:
        X.SibSp = X.SibSp.divide(8)

    if "Parch" in X:
        X.Parch = X.Parch.divide(6)

    if "Fare" in X:
        # Since only a few would miss 'fare' value, it's okay to fill with average.
        X.Fare = X.Fare.fillna(X.Fare.mean()).divide(512)

    if "Embarked" in X:
        X.Embarked = X.Embarked.astype(pd.api.types.CategoricalDtype(categories="C Q S".split()))
        X = pd.get_dummies(X, columns=["Embarked"])

    if "Name" in X:
        X["Title"] = X.Name.apply(lambda name: re.search(", ([\w ]+).", name).group(1))

        # Try to see if the person is married (1), or not (-1), or unknown (0).
        X["Married"] = X.Title.apply(lambda title: 1 if title in "Mrs Mr".split() else -1 if title in "Miss Master".split() else 0)

        # Get dummies for title, but include all possible values, even those not present in current dataset.
        X.Title = X.Title.astype(pd.api.types.CategoricalDtype(categories=titles))
        X = pd.get_dummies(X, columns=["Title"])
        
    if "Cabin" in X:
        X.Cabin = X.Cabin.fillna("-")
        
        X["Cabin_Number"] = X.Cabin.apply(lambda cabin: int(re.search("\w(\d+)", cabin).group(1)) if len(cabin) > 1 else 0)
        # Do some sort of normalization.
        X.Cabin_Number = X.Cabin_Number.divide(200)
        X["Cabin_Number_Missing"] = np.where(X.Cabin_Number == 0, 1, 0)
        
        X.Cabin = X.Cabin.apply(lambda cabin: cabin[:1])
        
        X.Cabin = X.Cabin.astype(pd.api.types.CategoricalDtype(categories=list("ABCDEFGT")))
        X = pd.get_dummies(X, columns=["Cabin"], dummy_na=True)

    if "Age" in X:
        X["Age_Missing"] = np.where(X.Age.isnull(), 1, 0)

        # No need to skip 'nan' for Age when calculating mean, as Pandas does that automatically.
        # 'transform' will go through each group, and fill its nan values with its mean value.
        # Then, all that will be aggregated back into the column, thus replacing nan values with group's mean.
        X["Age"] = X.groupby("Pclass Sex".split())["Age"].transform(lambda x: x.fillna(x.mean()))
        
        X.Age = X.Age.divide(80)

    # Needs to be after 'Age', since age is using original Sex column.
    if "Sex" in X:
        X.Sex = X.Sex.astype(pd.api.types.CategoricalDtype(categories="male female".split()))
        X = pd.get_dummies(X, columns=["Sex"])
        
    # = Remove extra features = 
    remove_features = "Name".split()
    X = X.drop(columns=remove_features)
    
    # === Get Y - the result values. ===
    Y = csv_data.get("Survived")
    
    print(X.head(5))
    print(None if Y is None else Y.head(5))
    
    return np.array(X), np.array(Y)

In [6]:
print("Train data:")
train_data_X, train_data_Y = prepare_data(train_data)

Train data:
     Pclass  SibSp  Parch      Fare     Age  Embarked_C  Embarked_Q  Embarked_S  Married  Title_Mr  ...  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  Cabin_nan  Age_Missing  Sex_male  Sex_female
0  1.000000  0.125    0.0  0.014160  0.2750           0           0           1        1         1  ...        0        0        0        0        0        0          1            0         1           0
1  0.333333  0.125    0.0  0.139225  0.4750           1           0           0        1         0  ...        1        0        0        0        0        0          0            0         0           1
2  1.000000  0.000    0.0  0.015479  0.3250           0           0           1       -1         0  ...        0        0        0        0        0        0          1            0         0           1
3  0.333333  0.125    0.0  0.103711  0.4375           0           0           1        1         0  ...        1        0        0        0        0        0          0    

In [7]:
print("Test data:")
test_data_X, _ = prepare_data(test_data)

Test data:
     Pclass  SibSp     Parch      Fare      Age  Embarked_C  Embarked_Q  Embarked_S  Married  Title_Mr  ...  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  Cabin_nan  Age_Missing  Sex_male  Sex_female
0  1.000000  0.000  0.000000  0.015291  0.43125           0           1           0        1         1  ...        0        0        0        0        0        0          1            0         1           0
1  1.000000  0.125  0.000000  0.013672  0.58750           0           0           1        1         0  ...        0        0        0        0        0        0          1            0         0           1
2  0.666667  0.000  0.000000  0.018921  0.77500           0           1           0        1         1  ...        0        0        0        0        0        0          1            0         1           0
3  1.000000  0.000  0.000000  0.016919  0.33750           0           0           1        1         1  ...        0        0        0        0        0     

In [8]:
from sklearn.model_selection import train_test_split

train_X, dev_X, train_Y, dev_Y = train_test_split(train_data_X, train_data_Y, test_size=0.15, random_state=42)

train_Y = train_Y.reshape((-1, 1))
dev_Y = dev_Y.reshape((-1, 1))

print(train_X.shape, train_Y.shape)
print(dev_X.shape, dev_Y.shape)

(757, 41) (757, 1)
(134, 41) (134, 1)


In [9]:
# Uncomment for the submission - use whole data set for training.

# train_X, train_Y = train_data_X, train_data_Y

In [10]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.1.0


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

def get_model(input_size):
    model = Sequential([
        Dense(128, input_shape=(input_size,), activation="relu", kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        Dense(64, activation="relu", kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        Dense(32, activation="relu", kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        Dense(16, activation="relu", kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        Dense(1, activation="sigmoid")
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(3e-4), metrics=["accuracy"], loss="binary_crossentropy")
    
    return model

In [12]:
model = get_model(train_X.shape[1])

In [13]:
model.fit(train_X, train_Y, epochs=500, batch_size=32, validation_data=(dev_X, dev_Y))

Train on 757 samples, validate on 134 samples
Epoch 1/500
757/757 [==============================] - 1s 2ms/sample - loss: 2.7075 - accuracy: 0.5786 - val_loss: 2.5944 - val_accuracy: 0.7612
Epoch 2/500
757/757 [==============================] - 0s 96us/sample - loss: 2.5032 - accuracy: 0.6684 - val_loss: 2.3991 - val_accuracy: 0.7537
Epoch 3/500
757/757 [==============================] - 0s 91us/sample - loss: 2.3321 - accuracy: 0.6764 - val_loss: 2.2212 - val_accuracy: 0.7612
Epoch 4/500
757/757 [==============================] - 0s 88us/sample - loss: 2.1629 - accuracy: 0.7015 - val_loss: 2.0591 - val_accuracy: 0.7537
Epoch 5/500
757/757 [==============================] - 0s 87us/sample - loss: 2.0095 - accuracy: 0.7279 - val_loss: 1.9158 - val_accuracy: 0.8060
Epoch 6/500
757/757 [==============================] - 0s 87us/sample - loss: 1.8814 - accuracy: 0.7398 - val_loss: 1.7940 - val_accuracy: 0.7910
Epoch 7/500
757/757 [==============================] - 0s 91us/sample - loss: 1

In [14]:
_, accuracy = model.evaluate(train_X, train_Y, verbose=2)

print('\nTrain accuracy:', accuracy)

757/757 - 0s - loss: 0.4416 - accuracy: 0.8534

Train accuracy: 0.8533686


In [15]:
_, accuracy = model.evaluate(dev_X, dev_Y, verbose=2)

print('\nDev accuracy:', accuracy)

134/134 - 0s - loss: 0.4753 - accuracy: 0.8134

Dev accuracy: 0.8134328


In [16]:
from kerastuner import HyperModel

class TitanicHyperModel(HyperModel):
    def __init__(self, input_size):
        self.input_shape = (input_size, )

    def build(self, hp):
        model = Sequential([
            Dense(
                units=hp.Int(
                    "dense_units_1",
                    min_value=64,
                    max_value=128,
                    step=64,
                    default=64
                ),
                input_shape=self.input_shape,
                activation=hp.Choice(
                    'dense_activation_1',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            ),
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            )),
            Dense(
                units=hp.Int(
                    "dense_units_2",
                    min_value=16,
                    max_value=64,
                    step=16,
                    default=32
                ),
                activation=hp.Choice(
                    'dense_activation_2',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            ),
            Dropout(rate=hp.Float(
                'dropout_2',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            )),
            Dense(
                units=hp.Int(
                    "dense_units_3",
                    min_value=4,
                    max_value=32,
                    step=4,
                    default=16
                ),
                activation=hp.Choice(
                    'dense_activation_3',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            ),
            Dropout(rate=hp.Float(
                'dropout_3',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            )),
            Dense(
                units=hp.Int(
                    "dense_units_4",
                    min_value=4,
                    max_value=16,
                    step=2,
                    default=8
                ),
                activation=hp.Choice(
                    'dense_activation_4',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            ),
            Dropout(rate=hp.Float(
                'dropout_4',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            )),
            Dense(1, activation="sigmoid")
        ])
        
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            metrics=["accuracy"],
            loss="binary_crossentropy",
        )

        return model

hypermodel = TitanicHyperModel(input_size=train_X.shape[1])

In [17]:
SEED = 42
MAX_TRIALS = 200
EXECUTION_PER_TRIAL = 2
N_EPOCH_SEARCH = 500

In [18]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(
    hypermodel,
    objective="val_accuracy",
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='random_search_1',
    project_name='titanic'
)

In [19]:
tuner.search_space_summary()

In [20]:
tuner.search(train_X, train_Y, epochs=N_EPOCH_SEARCH, validation_data=(dev_X, dev_Y))

Train on 757 samples, validate on 134 samples
Epoch 1/500
757/757 [==============================] - 1s 1ms/sample - loss: 0.5483 - accuracy: 0.7173 - val_loss: 0.4731 - val_accuracy: 0.7910
Epoch 2/500
757/757 [==============================] - 0s 89us/sample - loss: 0.5157 - accuracy: 0.7768 - val_loss: 0.5000 - val_accuracy: 0.7910
Epoch 3/500
757/757 [==============================] - 0s 95us/sample - loss: 0.5064 - accuracy: 0.7873 - val_loss: 0.4721 - val_accuracy: 0.7761
Epoch 4/500
757/757 [==============================] - 0s 129us/sample - loss: 0.4714 - accuracy: 0.7939 - val_loss: 0.4418 - val_accuracy: 0.8209
Epoch 5/500
757/757 [==============================] - 0s 106us/sample - loss: 0.4566 - accuracy: 0.8164 - val_loss: 0.4219 - val_accuracy: 0.8433
Epoch 6/500
757/757 [==============================] - 0s 89us/sample - loss: 0.4594 - accuracy: 0.8111 - val_loss: 0.4344 - val_accuracy: 0.8284
Epoch 7/500
757/757 [==============================] - 0s 96us/sample - loss:

Train on 757 samples, validate on 134 samples
Epoch 1/500
757/757 [==============================] - 1s 1ms/sample - loss: 0.6999 - accuracy: 0.5310 - val_loss: 0.6810 - val_accuracy: 0.5821
Epoch 2/500
757/757 [==============================] - 0s 97us/sample - loss: 0.6762 - accuracy: 0.5931 - val_loss: 0.6771 - val_accuracy: 0.5821
Epoch 3/500
757/757 [==============================] - 0s 88us/sample - loss: 0.6622 - accuracy: 0.6050 - val_loss: 0.6714 - val_accuracy: 0.5821
Epoch 4/500
757/757 [==============================] - 0s 90us/sample - loss: 0.6638 - accuracy: 0.6050 - val_loss: 0.6614 - val_accuracy: 0.5821
Epoch 5/500
757/757 [==============================] - 0s 95us/sample - loss: 0.6635 - accuracy: 0.6116 - val_loss: 0.6517 - val_accuracy: 0.5821
Epoch 6/500
757/757 [==============================] - 0s 87us/sample - loss: 0.6656 - accuracy: 0.6169 - val_loss: 0.6441 - val_accuracy: 0.5821
Epoch 7/500
757/757 [==============================] - 0s 87us/sample - loss: 0

Train on 757 samples, validate on 134 samples
Epoch 1/500
757/757 [==============================] - 1s 1ms/sample - loss: 0.6311 - accuracy: 0.6380 - val_loss: 0.5153 - val_accuracy: 0.7910
Epoch 2/500
757/757 [==============================] - 0s 119us/sample - loss: 0.5248 - accuracy: 0.7596 - val_loss: 0.4721 - val_accuracy: 0.8060
Epoch 3/500
757/757 [==============================] - 0s 99us/sample - loss: 0.4909 - accuracy: 0.7979 - val_loss: 0.4775 - val_accuracy: 0.7836
Epoch 4/500
757/757 [==============================] - 0s 99us/sample - loss: 0.4816 - accuracy: 0.8005 - val_loss: 0.4720 - val_accuracy: 0.8060
Epoch 5/500
757/757 [==============================] - 0s 116us/sample - loss: 0.4973 - accuracy: 0.7979 - val_loss: 0.4498 - val_accuracy: 0.8134
Epoch 6/500
757/757 [==============================] - 0s 100us/sample - loss: 0.4795 - accuracy: 0.7952 - val_loss: 0.4430 - val_accuracy: 0.8134
Epoch 7/500
757/757 [==============================] - 0s 118us/sample - los

Train on 757 samples, validate on 134 samples
Epoch 1/500
757/757 [==============================] - 1s 1ms/sample - loss: 0.7450 - accuracy: 0.4161 - val_loss: 0.7097 - val_accuracy: 0.4179
Epoch 2/500
757/757 [==============================] - 0s 97us/sample - loss: 0.7160 - accuracy: 0.4557 - val_loss: 0.6913 - val_accuracy: 0.4179
Epoch 3/500
757/757 [==============================] - 0s 101us/sample - loss: 0.6877 - accuracy: 0.5376 - val_loss: 0.6785 - val_accuracy: 0.5821
Epoch 4/500
757/757 [==============================] - 0s 88us/sample - loss: 0.6821 - accuracy: 0.5720 - val_loss: 0.6689 - val_accuracy: 0.5821
Epoch 5/500
757/757 [==============================] - 0s 85us/sample - loss: 0.6700 - accuracy: 0.5852 - val_loss: 0.6599 - val_accuracy: 0.5821
Epoch 6/500
757/757 [==============================] - 0s 87us/sample - loss: 0.6637 - accuracy: 0.5945 - val_loss: 0.6522 - val_accuracy: 0.5821
Epoch 7/500
757/757 [==============================] - 0s 86us/sample - loss: 

In [21]:
tuner.results_summary()

In [22]:
tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

{'dense_units_1': 64,
 'dense_activation_1': 'tanh',
 'dropout_1': 0.1,
 'dense_units_2': 32,
 'dense_activation_2': 'relu',
 'dropout_2': 0.35000000000000003,
 'dense_units_3': 20,
 'dense_activation_3': 'sigmoid',
 'dropout_3': 0.4,
 'dense_units_4': 4,
 'dense_activation_4': 'sigmoid',
 'dropout_4': 0.0,
 'learning_rate': 0.003225943238477045}

In [23]:
print("Best model")

# Get the best model tuned.
model = tuner.get_best_models(num_models=1)[0]

_, accuracy = model.evaluate(train_X, train_Y)
print(f"Tuned train accuracy: {accuracy}")

_, accuracy = model.evaluate(dev_X, dev_Y)
print(f"Tuned dev accuracy: {accuracy}")

Best model
757/757 [==============================] - 0s 188us/sample - loss: 0.3301 - accuracy: 0.8785
Tuned train accuracy: 0.8784676194190979
134/134 [==============================] - 0s 86us/sample - loss: 0.4013 - accuracy: 0.8731
Tuned dev accuracy: 0.8731343150138855


In [24]:
predictions = model.predict(test_data_X)

predictions = np.round(predictions).astype(np.uint8).reshape((-1))

print(predictions)

[0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0
 0 1 0 1 1 0 0 1 0 0 1]


In [25]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('dl_submission.csv', index=False)